In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Membaca data training dan testing menggunakan pandas
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Mengubah label 'ham' dan 'spam' menjadi nilai numerik (0 dan 1)
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Menampilkan 5 baris pertama dari data training untuk verifikasi
print("Data Training:")
print(train_df.head())

In [ ]:
# Memisahkan pesan dan label
train_labels = train_df['label'].values
train_messages = train_df['message'].values
test_labels = test_df['label'].values
test_messages = test_df['message'].values

# Inisialisasi Tokenizer untuk mengubah teks menjadi urutan angka
vocab_size = 1000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token="<OOV>")

# Membuat vocabulary berdasarkan data training
tokenizer.fit_on_texts(train_messages)

# Mengubah setiap pesan menjadi urutan angka (sequences)
train_sequences = tokenizer.texts_to_sequences(train_messages)
test_sequences = tokenizer.texts_to_sequences(test_messages)

# Melakukan padding agar setiap sequence memiliki panjang yang sama
max_length = 120
train_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

# Menampilkan contoh hasil padding
print("\nContoh pesan asli:\n", train_messages[0])
print("\nContoh pesan setelah diubah menjadi sequence angka:\n", train_sequences[0])
print("\nContoh pesan setelah di-padding:\n", train_padded[0])

In [ ]:
vocab_size = 2000
embedding_dim = 64
max_length = 120


tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_messages)
train_sequences = tokenizer.texts_to_sequences(train_messages)
train_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_sequences = tokenizer.texts_to_sequences(test_messages)
test_padded = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')


# Arsitektur Model Final yang Lebih Kuat
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # Bidirectional LSTM dapat menangkap konteks dari kedua arah
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()



In [ ]:
num_epochs = 20
history = model.fit(train_padded,
                    train_labels,
                    epochs=num_epochs,
                    validation_data=(test_padded, test_labels),
                    verbose=2)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # Preprocess pesan input sama seperti data training
  sequence = tokenizer.texts_to_sequences([pred_text])
  padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')

  # Lakukan prediksi dengan model untuk mendapatkan probabilitas
  prediction_prob = model.predict(padded_sequence)[0][0]

  # Tentukan label berdasarkan ambang batas 0.5
  label = "spam" if prediction_prob > 0.5 else "ham"

  prediction = [prediction_prob, label]

  return (prediction)

# Uji fungsi dengan contoh pesan
pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
